<a href="https://colab.research.google.com/github/MaureenGatu/AUTOLIB-CAR-SHARE-DATAANALYSIS/blob/main/Week_4_IP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Importing Python Libralies**

In [2]:
import pandas as pd
import numpy as np

# **Uploading the dataset**

In [3]:
#Upload table
df = pd.read_csv('/content/Autolib_dataset (2).csv')


In [4]:
#previewing the table
df.head(5)

,Address,Cars,Bluecar counter,Utilib counter,Utilib 1.4 counter,Charge Slots,Charging Status,City,Displayed comment,ID,Kind,Geo point,Postal code,Public name,Rental status,Scheduled at,Slots,Station type,Status,Subscription status,year,month,day,hour,minute
0,2 Avenue de Suffren,0,0,0,0,0,nonexistent,Paris,NaN,paris-suffren-2,STATION,"48.857, 2.2917",75015,Paris/Suffren/2,operational,NaN,2,station,ok,nonexistent,2018,4,8,11,43
1,145 Rue Raymond Losserand,6,6,0,0,0,operational,Paris,NaN,paris-raymondlosserand-145,STATION,"48.83126, 2.313088",75014,Paris/Raymond Losserand/145,operational,NaN,0,station,ok,nonexistent,2018,4,6,7,24
2,2 Avenue John Fitzgerald Kennedy,3,3,0,2,0,operational,Le Bourget,NaN,lebourget-johnfitzgeraldkennedy-2,STATION,"48.938103, 2.4286035",93350,Le Bourget/John Fitzgerald Kennedy/2,operational,NaN,1,station,ok,nonexistent,2018,4,3,20,14
3,51 Rue EugÃ¨ne OudinÃ©,3,3,1,0,1,operational,Paris,NaN,paris-eugeneoudine-51,STATION,"48.8250327, 2.3725162",75013,Paris/EugÃ¨ne OudinÃ©/51,operational,NaN,2,station,ok,nonexistent,2018,4,4,4,37
4,6 avenue de la Porte de Champerret,3,3,0,0,0,nonexistent,Paris,NaN,paris-portedechamperret-6,PARKING,"48.8862632, 2.2874511",75017,Paris/Porte de Champerret/6,operational,NaN,3,station,ok,nonexistent,2018,4,8,17,23


In [5]:
#shape of dataframe
df.shape

(5000, 25)

In [6]:
# retrieving information about our dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 25 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Address              5000 non-null   object
 1   Cars                 5000 non-null   int64 
 2   Bluecar counter      5000 non-null   int64 
 3   Utilib counter       5000 non-null   int64 
 4   Utilib 1.4 counter   5000 non-null   int64 
 5   Charge Slots         5000 non-null   int64 
 6   Charging Status      5000 non-null   object
 7   City                 5000 non-null   object
 8   Displayed comment    111 non-null    object
 9   ID                   5000 non-null   object
 10  Kind                 5000 non-null   object
 11  Geo point            5000 non-null   object
 12  Postal code          5000 non-null   int64 
 13  Public name          5000 non-null   object
 14  Rental status        5000 non-null   object
 15  Scheduled at         47 non-null     object
 16  Slots 

# **Data Cleaning**

Data Validation

In [7]:
#Drop Geo point as it not used for the study
#Drop Scheduled and Displayed comments as they ere not required for the study 
#also have a lot of missing information
#Drop cars, the column is redundant since cars is always equal to Bluecar Counter
to_drop = ['Geo point','Scheduled at','Displayed comment','Cars', 'Address','ID','minute']
df.drop(to_drop, axis = 1,inplace = True)

In [8]:
#preveiwing the table
df.head(5)

,Bluecar counter,Utilib counter,Utilib 1.4 counter,Charge Slots,Charging Status,City,Kind,Postal code,Public name,Rental status,Slots,Station type,Status,Subscription status,year,month,day,hour
0,0,0,0,0,nonexistent,Paris,STATION,75015,Paris/Suffren/2,operational,2,station,ok,nonexistent,2018,4,8,11
1,6,0,0,0,operational,Paris,STATION,75014,Paris/Raymond Losserand/145,operational,0,station,ok,nonexistent,2018,4,6,7
2,3,0,2,0,operational,Le Bourget,STATION,93350,Le Bourget/John Fitzgerald Kennedy/2,operational,1,station,ok,nonexistent,2018,4,3,20
3,3,1,0,1,operational,Paris,STATION,75013,Paris/EugÃ¨ne OudinÃ©/51,operational,2,station,ok,nonexistent,2018,4,4,4
4,3,0,0,0,nonexistent,Paris,PARKING,75017,Paris/Porte de Champerret/6,operational,3,station,ok,nonexistent,2018,4,8,17


In [9]:
#shape of dataframe
df.shape

(5000, 18)

In [10]:
#merge year month and date in to one date column
cols=["year","month","day"]
df['date'] = df[cols].apply(lambda x: '-'.join(x.values.astype(str)), axis="columns")
#set the new date column to the right date and time pandas format
df['date']=pd.to_datetime(df['date'])
#we can now drop the year month and day columns from the table.
to_drop = ['year','month', 'day']
df.drop(to_drop, axis = 1,inplace = True)

In [11]:
#view table info to confirm changes
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Bluecar counter      5000 non-null   int64         
 1   Utilib counter       5000 non-null   int64         
 2   Utilib 1.4 counter   5000 non-null   int64         
 3   Charge Slots         5000 non-null   int64         
 4   Charging Status      5000 non-null   object        
 5   City                 5000 non-null   object        
 6   Kind                 5000 non-null   object        
 7   Postal code          5000 non-null   int64         
 8   Public name          5000 non-null   object        
 9   Rental status        5000 non-null   object        
 10  Slots                5000 non-null   int64         
 11  Station type         5000 non-null   object        
 12  Status               5000 non-null   object        
 13  Subscription status  5000 non-nul

In [12]:
#shape of our data table
df.shape

(5000, 16)

2.Consistency

In [13]:
#check for duplicates
df.duplicated().unique()


array([False,  True])

In [14]:
# we now drop the duplicates
df = df.drop_duplicates()

In [15]:
#review shape after dropping duplicates
df.shape

(4974, 16)

Completeness of data

In [16]:
#Check for missing values

df.isna().any(axis=1).unique()

#there are missing values

array([False])

In [17]:
df.shape

(4974, 16)

Uniformity{column names)

In [18]:
#change the column names to conform to pascal naming convection
df=df.rename(columns={'Bluecar counter':'Bluecar Counter','Utilib counter':'Utilib Counter','Utilib 1.4 counter':'Utilib 1.4 Counter','Postal code':'Postal Code','Public name':'Public Name','Station type':'Station Type','Subscription status':'Subscription Status','hour':'Hour','date':'Date'})
df.head()

,Bluecar Counter,Utilib Counter,Utilib 1.4 Counter,Charge Slots,Charging Status,City,Kind,Postal Code,Public Name,Rental status,Slots,Station Type,Status,Subscription Status,Hour,Date
0,0,0,0,0,nonexistent,Paris,STATION,75015,Paris/Suffren/2,operational,2,station,ok,nonexistent,11,2018-04-08
1,6,0,0,0,operational,Paris,STATION,75014,Paris/Raymond Losserand/145,operational,0,station,ok,nonexistent,7,2018-04-06
2,3,0,2,0,operational,Le Bourget,STATION,93350,Le Bourget/John Fitzgerald Kennedy/2,operational,1,station,ok,nonexistent,20,2018-04-03
3,3,1,0,1,operational,Paris,STATION,75013,Paris/EugÃ¨ne OudinÃ©/51,operational,2,station,ok,nonexistent,4,2018-04-04
4,3,0,0,0,nonexistent,Paris,PARKING,75017,Paris/Porte de Champerret/6,operational,3,station,ok,nonexistent,17,2018-04-08


#**Test for Accuracy**

EXPECTATION

1.charge slots can only be greater than 0 when charging status    is operational

In [19]:
df['Charging Status'].unique()

array(['nonexistent', 'operational', 'broken'], dtype=object)

In [20]:
# EXPECTATION
#charge slots can only be greater than 0 when charging status is operational

#To test this expection, set the following condition.
#these conditions checks whether there is any row where charge slot is greater than zero
#   when the charging status is in any other status other than operational
if (df['Charge Slots'] > 0).all() and (df['Charging Status'] != 'operational').all():
  print('expectation not met')
#if there's any row where charge slot is greater than zero and charging status is anything but operational
#it should return 'true' meaning that the data does not meet the expctation
else:
  print('expectation was met')


expectation was met


2.slots and vehicles can be available in all situations (except future stations that have 0 resources)

In [21]:
#if there is any row where charging status is 'future' and it has the bluecar resorce then our 
#data is not accurate.

if (df['Charging Status'] == 'future').all() and (df['Bluecar Counter'] > 0).all():
  print('expectation not met')

#if no such row exixst, then the condition is satisfied and the data is accurate in this regard.
else:
  print('Expectation was met')


Expectation was met


In [22]:
#if there is any row where charging status is 'future' and it has the Utilib resorce then our 
#data is not accurate.

if (df['Charging Status'] == 'future').all() and (df['Utilib Counter'] > 0).all():
  print('expectation not met')
#if no such row exixst, then the condition is satisfied and the data is accurate in this regard.
else:
  print('Expectation was met')


Expectation was met


In [23]:
#if there is any row where charging status is 'future' and it has the Utilib 1.4 resorce then our 
#data is not accurate.

if (df['Charging Status'] == 'future').all() and (df['Utilib 1.4 Counter'] > 0).all():
  print('expectation not met')
#if no such row exixst, then the condition is satisfied and the data is accurate in this regard.
else:
  print('Expectation was met')

Expectation was met


In [24]:
#if there is any row where charging status is 'future' and it has the parking slots resorce then our 
#data is not accurate.

if (df['Charging Status'] == 'future').all() and (df['Slots'] > 0).all():
  print('expectation not met')
#if no such row exixst, then the condition is satisfied and the data is accurate in this regard.
else:
  print('Expectation was met')

Expectation was met


3.No resources available for subs_center which is just one location.

In [25]:
#if there is any row where station type is 'sub-center' and it has the Utilib 1.4 resorce then our 
#data is not accurate.

if (df['Station Type'] == 'sub_center').all() and (df['Utilib 1.4 Counter'] > 0).all():
  print('expectation not met')
#if no such row exixst, then the condition is satisfied and the data is accurate in this regard.
else:
  print('Expectation was met')

Expectation was met


In [26]:
#if there is any row where station type is 'sub-center' and it has the Utilib resorce then our 
#data is not accurate.

if (df['Station Type'] == 'sub_center').all() and (df['Utilib Counter'] > 0).all():
  print('expectation not met')
#if no such row exixst, then the condition is satisfied and the data is accurate in this regard.
else:
  print('Expectation was met')

Expectation was met


In [27]:
#if there is any row where station type is 'sub-center' and it has the Bluecar resorce then our 
#data is not accurate.

if (df['Station Type'] == 'sub_center').all() and (df['Bluecar Counter'] > 0).all():
  print('expectation not met')
#if no such row exixst, then the condition is satisfied and the data is accurate in this regard.
else:
  print('Expectation was met')

Expectation was met


In [28]:
#if there is any row where station type is 'sub-center' and it has the Slots resorce then our 
#data is not accurate.

if (df['Station Type'] == 'sub_center').all() and (df['Slots'] > 0).all():
  print('expectation not met')
#if no such row exixst, then the condition is satisfied and the data is accurate in this regard.
else:
  print('Expectation was met')

Expectation was met


4. No resourcses availble if Status is Scheduled

In [29]:
#status options
df['Status'].unique()

array(['ok', 'closed', 'scheduled'], dtype=object)

In [30]:
#no resources available when status is scheduled
#slots resource
if (df['Status'] == 'scheduled').all() and (df['Slots'] > 0).all():
  print('expectation not met')
#if no such row exixst, then the condition is satisfied and the data is accurate in this regard.
else:
  print('Expectation was met')

Expectation was met


In [31]:
#Bluecar resource
if (df['Status'] == 'scheduled').all() and (df['Bluecar Counter'] > 0).all():
  print('expectation not met')
#if no such row exixst, then the condition is satisfied and the data is accurate in this regard.
else:
  print('Expectation was met')

Expectation was met


In [32]:
#Utilib resource
if (df['Status'] == 'scheduled').all() and (df['Utilib Counter'] > 0).all():
  print('expectation not met')
#if no such row exixst, then the condition is satisfied and the data is accurate in this regard.
else:
  print('Expectation was met')

Expectation was met


In [33]:
#Utilib 1.4 resource
if (df['Status'] == 'scheduled').all() and (df['Utilib 1.4 Counter'] > 0).all():
  print('expectation not met')
#if no such row exixst, then the condition is satisfied and the data is accurate in this regard.
else:
  print('Expectation was met')

Expectation was met


5. No vehicles availabe if kind is center

In [34]:
#values in kind column
df['Kind'].unique()

array(['STATION', 'PARKING', 'SPACE', 'CENTER'], dtype=object)

In [35]:
#Bluecar resource
if (df['Kind'] == 'CENTER').all() and (df['Bluecar Counter'] > 0).all():
  print('expectation not met')
#if no such row exixst, then the condition is satisfied and the data is accurate in this regard.
else:
  print('Expectation was met')

Expectation was met


In [36]:
#Utilib 1.4 resource
if (df['Kind'] == 'CENTER').all() and (df['Utilib 1.4 Counter'] > 0).all():
  print('expectation not met')
#if no such row exixst, then the condition is satisfied and the data is accurate in this regard.
else:
  print('Expectation was met')

Expectation was met


In [37]:
#Utilib  resource
if (df['Kind'] == 'CENTER').all() and (df['Utilib Counter'] > 0).all():
  print('expectation not met')
#if no such row exixst, then the condition is satisfied and the data is accurate in this regard.
else:
  print('Expectation was met')

Expectation was met


6. No charging slots and Utilib 1.4 when kind is parking

In [38]:
#Utilib 1.4 resource
if (df['Kind'] == 'STATION').all() and (df['Utilib 1.4 Counter'] > 2).all():
  print('expectation not met')
#if no such row exixst, then the condition is satisfied and the data is accurate in this regard.
else:
  print('Expectation was met')

Expectation was met


In [39]:
#Utilib 1.4 resource
if (df['Kind'] == 'PARKING').all() and (df['Charge Slots'] > 0).all():
  print('expectation not met')
#if no such row exixst, then the condition is satisfied and the data is accurate in this regard.
else:
  print('Expectation was met')

Expectation was met


# ANALYSIS

##Popular Hour to pick up Electric Vehicles

In [40]:
#First, we create a table index by hour that has the aggregate sum of each car type per hour for the month of april.
# the table is conditional oon when cars are available at the staions
df['cars']=df['Bluecar Counter'] + df['Utilib 1.4 Counter'] + df['Utilib Counter']
hour =df[(df['City']== 'Paris') & (df['Date']>='2018-04-01') & (df['Date']<='2018-04-30') & (df['Rental status']=='operational')& (df['Station Type'] != 'sub_center')& (df['Kind'] != 'center')]
hour_table =pd.pivot_table(hour,values=['Bluecar Counter','Utilib Counter','Utilib 1.4 Counter','cars'],index=['Hour'],aggfunc=np.sum)
hour_table

,Bluecar Counter,Utilib 1.4 Counter,Utilib Counter,cars
Hour,,,,
0,201,10,5,216
1,205,10,3,218
2,221,9,7,237
3,242,14,5,261
4,209,10,3,222
5,228,18,5,251
6,260,15,13,288
7,223,11,8,242
8,234,11,12,257


In [41]:
#to get the most popular hour to get a bluecar, sort the hour_table by bluecar counter
#in ascending order and display the fisrt hour.

hour_table['Bluecar Counter'].sort_values().head(1)


Hour
20    149
Name: Bluecar Counter, dtype: int64

In [42]:
#to get the most popular hour to get a Utilib  Counter, sort the hour_table by Utilib Counter'
#in ascending order and display the fisrt hour.

hour_table['Utilib Counter'].sort_values().head(1)


Hour
23    1
Name: Utilib Counter, dtype: int64

In [43]:
#to get the most popular hour to get a Utilib 1.4 Counter, sort the hour_table by bUtilib 1.4 Counter'
#in ascending order and display the fisrt hour.

hour_table['Utilib 1.4 Counter'].sort_values().head(1)

Hour
2    9
Name: Utilib 1.4 Counter, dtype: int64

In [44]:
#overall pick hour
hour_table['cars'].sort_values().head(1)

Hour
20    164
Name: cars, dtype: int64

1.Hour 2O(8:00pm) has the least number of cars at the stations, meaning most of the cars have already been picked. this shows that this is the overall most popular picking hour.
2. In the same analysis, the most popular hour for picking Utilibs is at 11:00pm which is when the Utilibs are least in the station.
3. Most popular hour for Utilib 1.4 2:00am

##Popular hour to return Electric vehivles

In [45]:
# What is the most popular hour for returning cars?
#the most popular hour for returning cars is when the parking slots availabe are least
car_retur = pd.pivot_table(df,values=['Slots'],index=['Hour'],aggfunc=np.sum)
car_retur.sort_values(by='Slots',ascending= True).head(3)

,Slots
Hour,
0,348
17,349
12,368


the most popular hour to return cars was at midnight which is when there were least available number of parking slots, showing that most of the cars were already parked. 5:00pm was the second most popular hour for returning vehicles followed by noon

##Popular Station & postal code

In [46]:
#most popular station overall
df.groupby('Public Name').count().sort_values(by = 'Bluecar Counter',ascending = False)['Bluecar Counter'].head(1)

Public Name
Paris/Porte de Montrouge/8    13
Name: Bluecar Counter, dtype: int64

In [47]:
#most popular station at the popular picking hour
s = df.loc[df['Hour'] == 21]['Public Name'].value_counts().max()
df.loc[s]['Public Name']

'Paris/EugÃ¨ne OudinÃ©/51'

In [48]:
#What postal code is the most popular for picking up Blue cars? 
#create a table of the Bluecar counter indexed by postal code and name of stations and order with the one
#with most bluecars to se the most popular station.
df.pivot_table(values='Bluecar Counter',index=['Postal Code','Public Name'],aggfunc=np.sum).sort_values(by='Bluecar Counter',ascending=False).head(1)

,,Bluecar Counter
Postal Code,Public Name,
92260,Fontenay-aux-Roses/Robert Marchand/32,50


In [49]:
#most popular sation for picking Utilibs
df.pivot_table(values='Utilib Counter',index=['Postal Code','Public Name'],aggfunc=np.sum).sort_values(by='Utilib Counter',ascending=False).head(1)

,,Utilib Counter
Postal Code,Public Name,
92360,Meudon/MarÃ©chal Juin/18,4


In [50]:
#most popular sation for picking Utilib 1.4
df.pivot_table(values='Utilib 1.4 Counter',index=['Postal Code','Public Name'],aggfunc=np.sum).sort_values(by='Utilib 1.4 Counter',ascending=False).head(1)

,,Utilib 1.4 Counter
Postal Code,Public Name,
92310,SÃ¨vres/Grande Rue/123,9


In [51]:
#Does the most popular station belong to the most popular  postal code?
df['cars']=df['Bluecar Counter'] + df['Utilib 1.4 Counter'] + df['Utilib Counter']
df.pivot_table(values=['cars'],index=['Public Name','Postal Code'],aggfunc=np.sum).sort_values(by='cars',ascending=0).head()

,,cars
Public Name,Postal Code,
Paris/Porte de Montrouge/8,75014,50
Fontenay-aux-Roses/Robert Marchand/32,92260,50
Le Plessis-Robinson/GalilÃ©e/16,92350,49
Fontenay-aux-Roses/Lombard/20,92260,48
Fontenay-aux-Roses/Division Leclerc/31,92260,46


In [52]:
#popular station at the most popular picking hour?
df2 = df[df['Hour'] == 20]
df2['cars']=df2['Bluecar Counter'] + df2['Utilib 1.4 Counter'] + df2['Utilib Counter']
df2.pivot_table(values=['cars'],index=['Hour','Public Name'],aggfunc=np.sum).sort_values(by='cars',ascending=0).head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


cars
Hour Public Name                                  
20   Le Blanc-Mesnil/Claude Terrasse/1          11
     Paris/Cambronne/Parking                    11
     CrÃ©teil/Gabriel FaurÃ©/18                 10
     Nanterre/Hoche/1                            7
     VerriÃ¨res-le-Buisson/MarÃ©chal Foch/56     7

1.the most popular stations are Paris/Porte de Montrouge/8 and Fontenay-aux-Roses/Robert Marchand/32 which both had the most number of cars for the month of april

2.the most popular stations at the overall popular picking hour are Le Blanc-Mesnil/Claude Terrasse/1	and Paris/Cambronne/Parking

3.the most popular station does belong to the most popular postal code

In [53]:
#Most popular city, top 3
df.groupby('City').count().sort_values(by = 'Bluecar Counter',ascending = False)['Bluecar Counter'].head(3)

City
Paris                  2801
Nanterre                125
Issy-les-Moulineaux     101
Name: Bluecar Counter, dtype: int64